In [1]:
import re, string, collections
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

#from wordcloud import WordCloud
#from nltk import SnowballStemmer
#from nltk.corpus import stopwords
#nltk.download('stopwords')
import pickle
import numpy as np

### Limpieza de Corpus
Obtenemos el corpus directamente del archivo, eliminamos todos los símbolos que no son útiles para el análisis:
~~~
}{¡"$%&\'()¿:=\+[\]*-
~~~
Dejamos los símbolos que nos servirán para separar frases como coma, punto y coma, admiración e interrogación. Adicionalmente sustituimos todas las entidades conocidas que son equivalentes entre sí como los hashtag, menciones y vínculos. 

In [2]:
def cleanData (file, symbols=''):
    tweets = pd.read_csv(file, header=0, names=['ID', 'tweet'], sep='\t')
    #tweets = pd.read_csv(r'train.tsv', sep='\t')
    data = tweets.get('tweet').tolist()
    clean_data = list()
    for tweet in data:
        url = re.sub(r'http\S+','', tweet)
        simbolos = re.sub(symbols, '', url)
        usuarios = re.sub(r'@\w+', '_user_', simbolos)
        hashtags = re.sub(r'#\w+', '_hashtag_', usuarios)
        clean_data.append(hashtags.lower())
    return data, clean_data

### Etiquetas de inicio y final de oraciones
Utilizamos los simbolos especiales para delimitar el inicio y fin de oraciones, posterior a esto se vuelven a limpiar las oraciones elimiando cualquier símbolo especial que haya podido quedar por faltas de ortografía.

In [3]:
def markData (data):
    frases_pattern = re.compile(
        "(\w+)( )?"
        "(\.|,|;|\?|!)+"
        "( )?(\w+)"
    )
    mark_data = list()
    for tweet in data:
        tweet = re.sub(frases_pattern, r'\1 _EOS_  _BOS_ \5', tweet, flags=0) #Lo deja para separa ideas
        tweet = ' _BOS_ ' + tweet + ' _EOS_ '
        tweet = re.sub('(\.|,|;|\?|!)+', r'', tweet, flags=0)
        mark_data.append(tweet)
    return mark_data

### Tokenización
Recorremos el corpus extrayendo cada uno de las palabras y etiquetas, generamos dos arreglos, en el primero se devuelve todo el corpus tokenizado y el segundo es un conjunto de todos los tokens únicos, es decir que solo aparecen una única ocasión en el corpus.

In [4]:
def createTokens(mark_data):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tokens = tokenizer.tokenize(mark_data)
    frequencies = collections.Counter(tokens)
    unique_tokens = [item[0] for item in filter(lambda x: x[1] == 1, frequencies.items())]
    return tokens, unique_tokens

### Creación del vocabulario
Crea un vocabulario basado en los tokens no únicos, obtenemos dos diccionarios que servirán para almacenar los índices asociados a cada token.

In [5]:
def createVocabulary (tokens, unique_tokens):
    vocabulario = list(set(tokens)-set(unique_tokens))
    size = len(vocabulario)
    vocabulario.append('_OOV_')
    dic_index = dict(zip(range(size),vocabulario))  #Índice -> Palabra
    dic_index[size] = '_OOV_' 
    dic_words = dict(zip(vocabulario, range(size))) #Palabra -> Índice
    dic_words['_OOV_'] = size
    return vocabulario, dic_index, dic_words

### Generación de pares de bigramas
Utiliza todos los tokens para generar los pares de bigramas, condicionamos a tener una frecuencia mayor a uno, cuando encuentra una palabra _out of vocabulary_ solamente agrega el segundo par de bigrama, es decir:
~~~
token_no_único -> OVV
~~~
De esta manera podemos calcular la probabilidad de que dado un token conocido se genere una palabra poco frecuente.

In [6]:
def genBigrams(dic_words,tokens,vocabulario):
    separador = (dic_words['_EOS_'],dic_words['_BOS_'])
    ovv = dic_words['_OOV_']
    tokens_idx = [dic_words[token] if token in vocabulario else ovv for token in tokens]
    #print(tokens_idx)
    bigrams = list()
    for i in range(len(tokens_idx) - 1):
        if tokens_idx[i] != ovv:
            bigrams.append((tokens_idx[i], tokens_idx[i+1]))
    return bigrams

## Ejecución del Pipeline

In [7]:
puntuacion = r'[\}\{¡"$%&\'()¿:=\+[\]*-]'
data, clean_data = cleanData(r'train_MX.tsv', puntuacion)
mark_data = markData(clean_data)
tokens, unique_tokens = createTokens(' '.join(mark_data))
vocabulario, dic_index, dic_words = createVocabulary(tokens, unique_tokens)
bigrams = genBigrams(dic_words,tokens,vocabulario)


### Resumen del proceso

In [8]:
print('Tamaño palabras únicas:     ',len(unique_tokens))
print('Tamaño palabras no únicas:  ',len(vocabulario)-1,'\n')


print('Bigramas para entrenamiento: ',len(bigrams),'\n')


tokenizer = nltk.RegexpTokenizer(r"\w+")
tokens_o = tokenizer.tokenize(' '.join(data))

print('Tokens originales en el corpus (sin preprocesamiento): ', len(tokens_o))
print('Tokens obtenidos agregando las etiquedas de inicio y final de oración :',len(tokens))

Tamaño palabras únicas:      3416
Tamaño palabras no únicas:   1728 

Bigramas para entrenamiento:  28129 

Tokens originales en el corpus (sin preprocesamiento):  25329
Tokens obtenidos agregando las etiquedas de inicio y final de oración : 31546


In [12]:
#Ver 66, 71, 114
def printOraciones(idx):
    print("Texto Original:\n",data[idx],"\n")
    print("Texto Limpio:\n",clean_data[idx],"\n")
    print("Texto Marcado:\n",mark_data[idx],"\n")
    
@interact
def navegacion(idx = [tweet for tweet in range(len(data))]):
    printOraciones(idx)


interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

#### Almacenamiento de resultados

In [8]:
#with open('Corpus.pk', 'wb') as corpus_file:
    pickle.dump(dic_words, corpus_file)
    pickle.dump(dic_index, corpus_file)
    pickle.dump(vocabulario, corpus_file)
    pickle.dump(bigrams, corpus_file)
    pickle.dump(tokens, corpus_file)
    pickle.dump(unique_tokens, corpus_file)